In [1]:
include("../src/SpinonStructure.jl")
include("../src/BZMath.jl")


using .BZmath
using .SpinonStructure
using StaticArrays
using SparseArrays
using LinearAlgebra
using BenchmarkTools
using Plots

In [16]:
using Pkg; Pkg.activate("../"); Pkg.add("Permutations")

  Activating project at `~/mygh/spinon-calc`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Permutations ─ v0.4.22
    Updating `~/mygh/spinon-calc/Project.toml`
  [2ae35dd2] + Permutations v0.4.22
    Updating `~/mygh/spinon-calc/Manifest.toml`
  [861a8166] + Combinatorics v1.0.2
  [2ae35dd2] + Permutations v0.4.22
Precompiling project...
  ✓ Permutations
  1 dependency successfully precompiled in 2 seconds. 174 already precompiled.


In [17]:
using Permutations

In [34]:
B = [0.02,0.04,0.07]

sim0 = SimulationParameters("0flux_prim",
    lattice=geom.PyroPrimitive(1,1,1),
    A=zeros(1,4),
    Jpm=-0.046,
    B=B,
    n_samples=10000
    )

sim211 = SimulationParameters("0flux_211",
    lattice=geom.PyroPrimitive(2,1,1),
    A=zeros(2,4),
    Jpm=-0.046,
    B=B,
    n_samples=10000
    )

sim511 = SimulationParameters("0flux_511",
    lattice=geom.PyroPrimitive(5,1,1),
    A=zeros(5,4),
    Jpm=-0.046,
    B=B,
    n_samples=10000
    )

sim222 = SimulationParameters("0flux_222",
    lattice=geom.PyroPrimitive(2,2,2),
    # A=[1 1 0 0; 0 0 -1 -1; 1 1 0 0; 1 1 0 0],
    A=zeros(2*2*2, 4),
    Jpm=-0.046,
    B=B,
    n_samples=10000
    )

@assert norm(calc_fluxes(sim2) - calc_fluxes(sim1)) < 1e-10
@assert norm(calc_fluxes(sim3) - calc_fluxes(sim1)) < 1e-10

In [45]:
cs0 = CompiledModel(sim0,0.2);
cs211 = CompiledModel(sim211,0.2);
cs511 = CompiledModel(sim511,0.2);

In [66]:
calcH(sim, Q) = Matrix(SpinonStructure.calc_hopping(sim, SVector{3}(Q)))

calcH (generic function with 1 method)

In [68]:
Q = 2π/8 .*[0.75,0.75,0]

H0 = calcH(cs0,Q)
H211 = calcH(cs211,Q)
H511 = calcH(cs511,Q)
P = Matrix( Permutation([2,3,4,5,1,7,8,9,10,6]) )
@assert norm(inv(P)*H511*(P) - H511) < 1e-14

In [79]:
e0, vec0 = eigen(H0)
e0x, vec0x = eigen(calcH(cs0, Q+geom.reciprocal_basis(cs211.sim.lat)[:,1]) )
e211, vec211 = eigen(Matrix(H211))
# e2, vec2 = eigen(Matrix(H0))
println("H0-> ",   e0   )
println("H0-> ",   e0x   )
println("H211-> ", e211 )

H0-> [0.005448215856604842, 0.07865943188171987]
H0-> [0.008083283949771683, 0.037916716050228325]
H211-> [0.005448215856604853, 0.008083283949771671, 0.037916716050228276, 0.07865943188171975]


In [83]:
vec211[:,1] 

4-element Vector{ComplexF64}:
 -0.47118394503976413 - 0.1672892403496536im
  -0.4711839450397648 - 0.1672892403496543im
 -0.49999999999999933 - 1.8691273701970121e-16im
  -0.5000000000000007 - 0.0im

In [84]:
vec0[:,1]

2-element Vector{ComplexF64}:
 0.6663547254476939 + 0.23658271254157284im
 0.7071067811865474 + 0.0im